In [1]:
import pandas as pd
import numpy as np

In [2]:
#this next iteration will try to score the different expression levels instead of just counting them. hopefully this leads to 
# a better metric for cBAF loss. I want to try an algorithm that would survey different weights for each gene, but thats
# a lot of effort and im tired

In [3]:
Omics_EPCG_TPMlogp1 = pd.read_csv('OmicsExpressionProteinCodingGenesTPMLogp1.csv')
Omics_EPCG_TPMlogp1 = Omics_EPCG_TPMlogp1.rename(columns={'Unnamed: 0': 'Depmap ID'})
Omics_EPCG_TPMlogp1.columns = Omics_EPCG_TPMlogp1.columns.str.split(' ').str[0]
Omics_EPCG_TPMlogp1.head()

,Depmap,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1
0,ACH-001113,4.331992,0.000000,7.364660,2.792855,4.471187,0.028569,1.226509,3.044394,6.500005,...,2.689299,0.189034,0.201634,2.130931,0.555816,0.0,0.275007,0.0,0.0,0.000000
1,ACH-001289,4.567424,0.584963,7.106641,2.543496,3.504620,0.000000,0.189034,3.813525,4.221877,...,1.286881,1.049631,0.321928,1.464668,0.632268,0.0,0.014355,0.0,0.0,0.000000
2,ACH-001339,3.150560,0.000000,7.379118,2.333424,4.228049,0.056584,1.310340,6.687201,3.682573,...,0.594549,1.097611,0.831877,2.946731,0.475085,0.0,0.084064,0.0,0.0,0.042644
3,ACH-001538,5.085340,0.000000,7.154211,2.545968,3.084064,0.000000,5.868390,6.165309,4.489928,...,0.214125,0.632268,0.298658,1.641546,0.443607,0.0,0.028569,0.0,0.0,0.000000
4,ACH-000242,6.729417,0.000000,6.537917,2.456806,3.867896,0.799087,7.208478,5.570159,7.127117,...,1.117695,2.358959,0.084064,1.910733,0.000000,0.0,0.464668,0.0,0.0,0.000000


In [4]:
print(len(Omics_EPCG_TPMlogp1))

1479


In [44]:
max_value = Omics_EPCG_TPMlogp1.iloc[:, 1:].to_numpy().max()

print(f"The largest number in the DataFrame is: {max_value}")

The largest number in the DataFrame is: 17.783669710269233


In [5]:
def calculate_25th_percentile(df):
    percentile_values = {}
    
    # Loop through each column in the DataFrame
    for column in df.columns:
        # Check if the column contains numeric data
        if pd.api.types.is_numeric_dtype(df[column]):
            # Filter out NaN and zero values
            non_zero_values = df[column][(df[column] != 0) & (df[column].notnull())]
            
            # Check if there are remaining values after filtering
            if len(non_zero_values) > 0:
                # Check if the column has more than one unique non-zero value
                if non_zero_values.nunique() > 1:
                    # Calculate the 75th percentile of the non-zero values
                    percentile_25 = np.percentile(non_zero_values, 25)
                    percentile_values[column] = percentile_25
                else:
                    percentile_values[column] = np.nan
            else:
                percentile_values[column] = np.nan
    
    return percentile_values

def calculate_average(df):
    avg_values = {}
    
    # Loop through each column in the DataFrame
    for column in df.columns:
        # Check if the column contains numeric data
        if pd.api.types.is_numeric_dtype(df[column]):
            # Filter out zeros and NaN values
            non_zero_values = df[column][(df[column] != 0) & (df[column].notnull())]
            
            # Check if there are remaining values after filtering
            if len(non_zero_values) > 0:
                # Check if the column has more than one unique non-zero value
                if non_zero_values.nunique() > 1:
                    # Calculate the average of the non-zero values
                    average = np.mean(non_zero_values)
                    avg_values[column] = average
                else:
                    avg_values[column] = np.nan
            else:
                avg_values[column] = np.nan
    
    return avg_values
def calculate_std (df):
    std_values = {}
    
    # Loop through each column in the DataFrame
    for column in df.columns:
        # Check if the column contains numeric data
        if pd.api.types.is_numeric_dtype(df[column]):
            # Check for missing values in the column
            if df[column].isnull().any():
                std_values[column] = np.nan
            else:
                # Check if the column has more than one unique value
                if df[column].nunique() > 1:
                    # Calculate the standard deviation of the column
                    std_dev = np.std(df[column])
                    std_values[column] = std_dev
                else:
                    std_values[column] = np.nan
    
    return std_values

In [6]:
RNA_per = calculate_25th_percentile(Omics_EPCG_TPMlogp1)
reference = pd.DataFrame(data=RNA_per, index=[0])
reference = reference.rename(index={0:'Average'})
reference = reference.transpose()
reference = reference.reset_index()
reference = reference.rename(columns={'index':'gene'})

In [8]:
cBAF_downreg = pd.read_excel('cBAF_downreg.xlsx')

In [18]:
ref2 = Omics_EPCG_TPMlogp1.transpose()
ref2 = ref2.reset_index()


row_index = 0
# Set the new header
new_header = ref2.iloc[row_index]  # Take the row to be used as header
df = ref2[1:]  # Remove the header row from the data
df.columns = new_header  # Set new header

# Reset index if desired
df.reset_index(drop=True, inplace=True)
df = df.rename(columns={'Depmap':'gene'})
df.head()

,gene,ACH-001113,ACH-001289,ACH-001339,ACH-001538,ACH-000242,ACH-000708,ACH-000327,ACH-000233,ACH-000461,...,ACH-001578,ACH-000036,ACH-000973,ACH-001128,ACH-000750,ACH-000285,ACH-002669,ACH-001858,ACH-001997,ACH-000052
0,TSPAN6,4.331992,4.567424,3.15056,5.08534,6.729417,4.27277,3.337711,0.056584,4.017031,...,6.345361,4.018812,4.328406,5.995032,3.533563,0.056584,3.111031,4.390943,5.05745,4.249445
1,TNMD,0.0,0.584963,0.0,0.0,0.0,0.189034,0.0,0.0,0.0,...,3.401903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DPM1,7.36466,7.106641,7.379118,7.154211,6.537917,7.023255,5.927659,6.094236,6.534497,...,7.211694,6.700856,7.059182,6.238978,6.488483,6.604368,7.031329,7.013239,7.815191,6.175724
3,SCYL3,2.792855,2.543496,2.333424,2.545968,2.456806,2.555816,1.944858,3.971773,2.226509,...,2.533563,2.137504,1.891419,2.304511,1.823749,3.266037,1.541019,1.887525,2.538538,2.31904
4,C1orf112,4.471187,3.50462,4.228049,3.084064,3.867896,3.841973,2.678072,3.731183,3.02148,...,4.374344,2.531069,3.529821,4.0,3.308885,4.973152,3.664483,3.252476,3.893362,3.825786


In [10]:
cBAF_score = pd.merge(df, reference, on = 'gene', how = 'inner')
cBAF_score = pd.merge(cBAF_score, cBAF_downreg, on = 'gene', how = 'inner')

In [30]:
cBAF_score.head()

,gene,ACH-001113,ACH-001289,ACH-001339,ACH-001538,ACH-000242,ACH-000708,ACH-000327,ACH-000233,ACH-000461,...,TCGA-COAD_qvalue,TCGA-LUAD_qvalue,TCGA-LUSC_qvalue,TCGA-BRCA_qvalue,TCGA-KIRP_qvalue,TCGA-READ_qvalue,TCGA-LIHC_qvalue,TCGA-BLCA_qvalue,annotated_functions,CheckResults
0,STAB1,0.056584,0.084064,0.070389,0.0,0.014355,0.137504,0.028569,0.042644,0.201634,...,0.622912,0.928117,0.999958,0.019496,3.577760e-08,0.937959,0.018408,0.003142,protein binding,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."
1,SLC6A7,0.0,0.0,0.0,0.042644,0.0,1.970854,0.014355,0.0,0.0,...,0.918358,0.015666,0.999958,0.032801,3.664958e-04,0.802462,0.029678,0.488808,None,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."
2,ALOX5,0.214125,0.214125,0.014355,0.62293,5.782933,1.944858,0.201634,0.238787,1.077243,...,0.678948,0.043900,0.999958,0.016656,9.481017e-03,0.569912,0.000022,0.067270,protein binding,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."
3,MTMR11,1.831877,4.831877,1.669027,3.778209,1.765535,3.91265,2.759156,2.970854,2.865919,...,0.613535,0.878440,0.999958,0.008080,5.340482e-01,0.006816,0.002723,0.000059,None,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."
4,ADAM28,0.111031,0.097611,0.124328,2.811471,0.978196,0.0,0.214125,1.819668,0.176323,...,0.049856,0.700369,0.999958,0.016491,3.649882e-02,0.992190,0.000002,0.124470,None,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."


In [11]:
def check_numbers(row):
    # 'Average' is the reference number for comparison
    defined_number = row['Average']
    
    # Initialize results dictionary
    results = {}
    
    # Iterate through each column except 'Average'
    for col in row.index:
        if col != 'Average':
            value = row[col]
            
            # Try to convert value to a float
            try:
                value = float(value)
                # Check if value is less than defined_number and not zero
                results[col] = (value < defined_number and value != 0)
            except ValueError:
                # If conversion fails, skip this value
                results[col] = False
    
    return results

In [12]:
# Apply the function to each row and store the results in a new column
cBAF_score['CheckResults'] = cBAF_score.apply(check_numbers, axis=1)

In [13]:
cBAF_score.head()

,gene,ACH-001113,ACH-001289,ACH-001339,ACH-001538,ACH-000242,ACH-000708,ACH-000327,ACH-000233,ACH-000461,...,TCGA-COAD_qvalue,TCGA-LUAD_qvalue,TCGA-LUSC_qvalue,TCGA-BRCA_qvalue,TCGA-KIRP_qvalue,TCGA-READ_qvalue,TCGA-LIHC_qvalue,TCGA-BLCA_qvalue,annotated_functions,CheckResults
0,STAB1,0.056584,0.084064,0.070389,0.0,0.014355,0.137504,0.028569,0.042644,0.201634,...,0.622912,0.928117,0.999958,0.019496,3.577760e-08,0.937959,0.018408,0.003142,protein binding,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."
1,SLC6A7,0.0,0.0,0.0,0.042644,0.0,1.970854,0.014355,0.0,0.0,...,0.918358,0.015666,0.999958,0.032801,3.664958e-04,0.802462,0.029678,0.488808,None,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."
2,ALOX5,0.214125,0.214125,0.014355,0.62293,5.782933,1.944858,0.201634,0.238787,1.077243,...,0.678948,0.043900,0.999958,0.016656,9.481017e-03,0.569912,0.000022,0.067270,protein binding,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."
3,MTMR11,1.831877,4.831877,1.669027,3.778209,1.765535,3.91265,2.759156,2.970854,2.865919,...,0.613535,0.878440,0.999958,0.008080,5.340482e-01,0.006816,0.002723,0.000059,None,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."
4,ADAM28,0.111031,0.097611,0.124328,2.811471,0.978196,0.0,0.214125,1.819668,0.176323,...,0.049856,0.700369,0.999958,0.016491,3.649882e-02,0.992190,0.000002,0.124470,None,"{'gene': False, 'ACH-001113': False, 'ACH-0012..."


In [15]:
#dictionary of gene, cell line tuple as key with element T or F if the expression should be counted towards the score
cBAF_score_dict = {}
for index, row in cBAF_score.iterrows():
    gene = row['gene']
    CheckResults = row['CheckResults']
    
    for string, boolean in CheckResults.items():
        key = (gene, string)
        cBAF_score_dict[key] = boolean

In [16]:
# Initialize a dictionary to hold the count of True values for each key
true_counts = {}

# Iterate over each row in the 'CheckResults' column
for results in cBAF_score['CheckResults']:
    for key, value in results.items():
        if value:  # If the value is True
            if key in true_counts:
                true_counts[key] += 1
            else:
                true_counts[key] = 1
counts = pd.DataFrame(list(true_counts.items()), columns=['Depmap ID','Count'])

In [21]:
# Replace 0s with NaN
df.replace(np.nan, 0, inplace=True)
df.head()

,gene,ACH-001113,ACH-001289,ACH-001339,ACH-001538,ACH-000242,ACH-000708,ACH-000327,ACH-000233,ACH-000461,...,ACH-001578,ACH-000036,ACH-000973,ACH-001128,ACH-000750,ACH-000285,ACH-002669,ACH-001858,ACH-001997,ACH-000052
0,TSPAN6,4.331992,4.567424,3.150560,5.085340,6.729417,4.272770,3.337711,0.056584,4.017031,...,6.345361,4.018812,4.328406,5.995032,3.533563,0.056584,3.111031,4.390943,5.057450,4.249445
1,TNMD,0.000000,0.584963,0.000000,0.000000,0.000000,0.189034,0.000000,0.000000,0.000000,...,3.401903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,DPM1,7.364660,7.106641,7.379118,7.154211,6.537917,7.023255,5.927659,6.094236,6.534497,...,7.211694,6.700856,7.059182,6.238978,6.488483,6.604368,7.031329,7.013239,7.815191,6.175724
3,SCYL3,2.792855,2.543496,2.333424,2.545968,2.456806,2.555816,1.944858,3.971773,2.226509,...,2.533563,2.137504,1.891419,2.304511,1.823749,3.266037,1.541019,1.887525,2.538538,2.319040
4,C1orf112,4.471187,3.504620,4.228049,3.084064,3.867896,3.841973,2.678072,3.731183,3.021480,...,4.374344,2.531069,3.529821,4.000000,3.308885,4.973152,3.664483,3.252476,3.893362,3.825786


In [22]:
df_numeric = df.select_dtypes(include=[float, int])  # Select numeric columns
df[df_numeric.columns] = 10 - df_numeric

In [23]:
df.head()

,gene,ACH-001113,ACH-001289,ACH-001339,ACH-001538,ACH-000242,ACH-000708,ACH-000327,ACH-000233,ACH-000461,...,ACH-001578,ACH-000036,ACH-000973,ACH-001128,ACH-000750,ACH-000285,ACH-002669,ACH-001858,ACH-001997,ACH-000052
0,TSPAN6,5.668008,5.432576,6.849440,4.914660,3.270583,5.727230,6.662289,9.943416,5.982969,...,3.654639,5.981188,5.671594,4.004968,6.466437,9.943416,6.888969,5.609057,4.942550,5.750555
1,TNMD,10.000000,9.415037,10.000000,10.000000,10.000000,9.810966,10.000000,10.000000,10.000000,...,6.598097,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
2,DPM1,2.635340,2.893359,2.620882,2.845789,3.462083,2.976745,4.072341,3.905764,3.465503,...,2.788306,3.299144,2.940818,3.761022,3.511517,3.395632,2.968671,2.986761,2.184809,3.824276
3,SCYL3,7.207145,7.456504,7.666576,7.454032,7.543194,7.444184,8.055142,6.028227,7.773491,...,7.466437,7.862496,8.108581,7.695489,8.176251,6.733963,8.458981,8.112475,7.461462,7.680960
4,C1orf112,5.528813,6.495380,5.771951,6.915936,6.132104,6.158027,7.321928,6.268817,6.978520,...,5.625656,7.468931,6.470179,6.000000,6.691115,5.026848,6.335517,6.747524,6.106638,6.174214


In [24]:
df.replace(10, np.nan, inplace=True)
df.head()

,gene,ACH-001113,ACH-001289,ACH-001339,ACH-001538,ACH-000242,ACH-000708,ACH-000327,ACH-000233,ACH-000461,...,ACH-001578,ACH-000036,ACH-000973,ACH-001128,ACH-000750,ACH-000285,ACH-002669,ACH-001858,ACH-001997,ACH-000052
0,TSPAN6,5.668008,5.432576,6.849440,4.914660,3.270583,5.727230,6.662289,9.943416,5.982969,...,3.654639,5.981188,5.671594,4.004968,6.466437,9.943416,6.888969,5.609057,4.942550,5.750555
1,TNMD,NaN,9.415037,NaN,NaN,NaN,9.810966,NaN,NaN,NaN,...,6.598097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DPM1,2.635340,2.893359,2.620882,2.845789,3.462083,2.976745,4.072341,3.905764,3.465503,...,2.788306,3.299144,2.940818,3.761022,3.511517,3.395632,2.968671,2.986761,2.184809,3.824276
3,SCYL3,7.207145,7.456504,7.666576,7.454032,7.543194,7.444184,8.055142,6.028227,7.773491,...,7.466437,7.862496,8.108581,7.695489,8.176251,6.733963,8.458981,8.112475,7.461462,7.680960
4,C1orf112,5.528813,6.495380,5.771951,6.915936,6.132104,6.158027,7.321928,6.268817,6.978520,...,5.625656,7.468931,6.470179,6.000000,6.691115,5.026848,6.335517,6.747524,6.106638,6.174214


In [28]:
# Initialize the result dictionary
values_to_sum = {}

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    gene = row['gene']
    
    # Iterate over each column (excluding 'gene')
    for column_name in df.columns[1:]:
        value = row[column_name]
        key = (gene, column_name)
        values_to_sum[key] = value


In [31]:
# Initialize an empty dictionary to store the sums
sum_dict = {}

# Iterate over result_dict to sum the values for each DepMap ID with True boolean
for (gene, depmap_id), boolean in cBAF_score_dict.items():
    if boolean:  # Only sum if the boolean is True
        if (gene, depmap_id) in values_to_sum:
            sum_dict[depmap_id] = sum_dict.get(depmap_id, 0) + values_to_sum[(gene, depmap_id)]
        else:
            print(f"Warning: Key ({gene}, {depmap_id}) not found in values_to_sum. Skipping...")



In [32]:
# Convert the sum_dict to a DataFrame
sum_df = pd.DataFrame(list(sum_dict.items()), columns=['Depmap ID', 'Sum'])
sum_df.head()

,Depmap ID,Sum
0,ACH-000242,285.170767
1,ACH-000327,335.874886
2,ACH-000233,177.753490
3,ACH-002040,216.760009
4,ACH-000769,246.840481


In [33]:
counts.head()

,Depmap ID,Count
0,ACH-000242,29
1,ACH-000327,34
2,ACH-000233,18
3,ACH-002040,22
4,ACH-000769,25


In [34]:
final_df = pd.merge(sum_df, counts, on = 'Depmap ID', how = 'inner')

In [35]:
final_df.head()

,Depmap ID,Sum,Count
0,ACH-000242,285.170767,29
1,ACH-000327,335.874886,34
2,ACH-000233,177.753490,18
3,ACH-002040,216.760009,22
4,ACH-000769,246.840481,25


In [36]:
final_df['Score'] = final_df['Sum'] * final_df['Count']

In [37]:
final_df.to_excel('cBAF loss score v8.xlsx')

In [38]:
DRC_table_all = pd.read_excel('DRCtableAUC_merge_DepmapID.xlsx')
DRC_table_all.head()

,Cell name,Depmap ID,"AUC, DEG-35","AUC, DEG-35 sigfig 4"
0,143B,ACH-001001,0.738025,0.7380
1,22RV1,ACH-000956,0.543824,0.5438
2,2313287,ACH-000948,0.362258,0.3623
3,253J,ACH-000011,0.811752,0.8118
4,253JBV,ACH-000026,0.713113,0.7131


In [39]:
cBAFscore_final_df = pd.merge(final_df, DRC_table_all, on = 'Depmap ID', how = 'inner')

In [40]:
cBAFscore_final_df.to_excel('cBAF_loss_score_v9.xlsx')

In [41]:
DRC_table_p53WT = pd.read_excel('TP53WT_cells_AUC_DepmapID.xlsx')

In [42]:
cBAFscore_p53WT = pd.merge(final_df,DRC_table_p53WT, on = 'Depmap ID', how = 'inner')

In [43]:
cBAFscore_p53WT.to_excel('cBAF_loss_score_v9_p53WT cells.xlsx')